# PREGRADO Course Discovery

**Objective:** Find high-potential courses for early failure prediction analysis in PREGRADO (excluding already-analyzed Control de Gestión Account 719).

**Criteria for High-Potential Courses:**
1. 20+ students (statistical significance)
2. Grade variance > 10 (standard deviation of final_score)
3. Pass rate 20-80% (class diversity for ML)
4. 5+ assignments (good LMS design)
5. Current or recent term (term_id 336 or 322)

---

## 1. Setup & API Connection

In [ ]:
import requests
import os
import time
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from tqdm.notebook import tqdm

# Load environment variables
load_dotenv()

API_URL = os.getenv('CANVAS_API_URL')
API_TOKEN = os.getenv('CANVAS_API_TOKEN')
headers = {'Authorization': f'Bearer {API_TOKEN}'}

# Configuration
PREGRADO_ID = 46
EXCLUDE_ACCOUNTS = [719]  # Already analyzed: Ing. en Control de Gestión
TARGET_TERMS = [336, 322]  # 2nd Sem 2025 (current) and 1st Sem 2025 (recent past)
MIN_STUDENTS = 15

print(f"API URL: {API_URL}")
print(f"Token configured: {'Yes' if API_TOKEN else 'No'}")

: 

In [ ]:
# Test API connection
r = requests.get(f'{API_URL}/api/v1/users/self', headers=headers)
if r.status_code == 200:
    user = r.json()
    print(f"Connected as: {user.get('name', 'Unknown')}")
    print(f"User ID: {user.get('id')}")
    print(f"Rate Limit Remaining: {r.headers.get('X-Rate-Limit-Remaining', 'N/A')}")
else:
    print(f"Connection failed: {r.status_code}")
    print(r.text)

## 2. Rate Limit Management

Canvas API has rate limits. We'll implement adaptive delays based on remaining quota.

In [ ]:
def calculate_delay(remaining_quota):
    """
    Calculate delay based on remaining API quota.
    More aggressive backing off as quota decreases.
    """
    if remaining_quota < 50:
        return 30  # Critical
    elif remaining_quota < 100:
        return 10  # Very low
    elif remaining_quota < 200:
        return 5   # Low
    elif remaining_quota < 300:
        return 2   # Moderate
    elif remaining_quota < 500:
        return 1   # Healthy
    else:
        return 0.5  # Abundant

def safe_request(url, headers, params=None, max_retries=3):
    """
    Make a request with rate limit handling.
    Returns (response_json, rate_limit_remaining) or (None, 0) on failure.
    """
    for attempt in range(max_retries):
        try:
            r = requests.get(url, headers=headers, params=params, timeout=30)
            remaining = int(float(r.headers.get('X-Rate-Limit-Remaining', 700)))
            
            if r.status_code == 403:
                print(f"Rate limited! Waiting 60s... (attempt {attempt + 1})")
                time.sleep(60)
                continue
            
            if r.status_code == 200:
                delay = calculate_delay(remaining)
                time.sleep(delay)
                return r.json(), remaining
            else:
                print(f"Error {r.status_code}: {r.text[:100]}")
                return None, remaining
                
        except Exception as e:
            print(f"Request failed (attempt {attempt + 1}): {e}")
            time.sleep(2 ** attempt)
    
    return None, 0

print("Rate limit functions defined.")

## 3. Discover PREGRADO Sub-Accounts

Get all sub-accounts under PREGRADO (Account 46), excluding Control de Gestión (719).

In [ ]:
# Get PREGRADO sub-accounts
sub_accounts_data, remaining = safe_request(
    f'{API_URL}/api/v1/accounts/{PREGRADO_ID}/sub_accounts',
    headers,
    params={'per_page': 100}
)

if sub_accounts_data:
    # Filter out already-analyzed accounts
    sub_accounts = [a for a in sub_accounts_data if a['id'] not in EXCLUDE_ACCOUNTS]
    
    print(f"Total PREGRADO sub-accounts: {len(sub_accounts_data)}")
    print(f"Unexplored sub-accounts: {len(sub_accounts)}")
    print(f"Rate limit remaining: {remaining}")
    print("\nSub-accounts to explore:")
    for acc in sub_accounts[:10]:
        print(f"  {acc['id']}: {acc['name']}")
    if len(sub_accounts) > 10:
        print(f"  ... and {len(sub_accounts) - 10} more")
else:
    print("Failed to fetch sub-accounts")
    sub_accounts = []

In [ ]:
# Create a DataFrame of sub-accounts for reference
if sub_accounts:
    sub_accounts_df = pd.DataFrame([
        {'account_id': a['id'], 'name': a['name'], 'parent_id': a.get('parent_account_id')}
        for a in sub_accounts
    ])
    display(sub_accounts_df.head(20))

## 4. Scan Sub-Accounts for Courses

We'll scan the top 3 sub-accounts (by size or priority) for courses with 15+ students.

In [ ]:
# Target sub-accounts from pregrado_discovery_plan.md + Providencia expansion
# These are the priority accounts to explore
TARGET_SUB_ACCOUNTS = [
    (730, "Ingeniería Civil Industrial"),
    (247, "Psicología"),
    (253, "Derecho"),
    (176, "Providencia"),  # Added - large sede with 43 careers, 3393 courses
]

# If TARGET_SUB_ACCOUNTS is empty, use first 3 from discovered list
if not TARGET_SUB_ACCOUNTS and sub_accounts:
    TARGET_SUB_ACCOUNTS = [(a['id'], a['name']) for a in sub_accounts[:3]]

print("Target sub-accounts for course scanning:")
for acc_id, name in TARGET_SUB_ACCOUNTS:
    print(f"  {acc_id}: {name}")

In [ ]:
def get_courses_from_account(account_id, term_ids=None, min_students=15):
    """
    Get courses from an account with minimum student count.
    Filters by enrollment term if specified.
    """
    all_courses = []
    
    for term_id in (term_ids or [None]):
        params = {
            'per_page': 100,
            'include[]': ['total_students', 'term'],
            'with_enrollments': True
        }
        if term_id:
            params['enrollment_term_id'] = term_id
        
        url = f'{API_URL}/api/v1/accounts/{account_id}/courses'
        
        while url:
            data, remaining = safe_request(url, headers, params)
            if not data:
                break
            
            for course in data:
                if course.get('total_students', 0) >= min_students:
                    all_courses.append({
                        'course_id': course['id'],
                        'name': course['name'],
                        'account_id': account_id,
                        'students': course.get('total_students', 0),
                        'term_name': course.get('term', {}).get('name', 'Unknown'),
                        'term_id': course.get('enrollment_term_id')
                    })
            
            # Check for pagination
            # Canvas uses Link header for pagination
            link_header = ''
            if hasattr(data, 'headers'):
                link_header = data.headers.get('Link', '')
            url = None  # Simple version - just get first page
            params = {}
    
    return all_courses

print("Course fetching function defined.")

In [ ]:
# Scan target sub-accounts for courses
all_candidate_courses = []

print("Scanning sub-accounts for courses...\n")

for acc_id, acc_name in tqdm(TARGET_SUB_ACCOUNTS, desc="Sub-accounts"):
    print(f"\nScanning: {acc_name} (ID: {acc_id})")
    
    courses = get_courses_from_account(acc_id, term_ids=TARGET_TERMS, min_students=MIN_STUDENTS)
    
    print(f"  Found {len(courses)} courses with {MIN_STUDENTS}+ students")
    
    all_candidate_courses.extend(courses)
    
    # Save progress
    if courses:
        for c in courses[:3]:
            print(f"    - {c['course_id']}: {c['name'][:40]}... ({c['students']} students)")

print(f"\n{'='*50}")
print(f"Total candidate courses: {len(all_candidate_courses)}")

In [ ]:
# Create DataFrame of candidate courses
if all_candidate_courses:
    courses_df = pd.DataFrame(all_candidate_courses)
    courses_df = courses_df.sort_values('students', ascending=False)
    
    print(f"Candidate courses by student count:")
    display(courses_df.head(20))
else:
    print("No candidate courses found.")
    courses_df = pd.DataFrame()

## 5. Analyze Course Potential

For each candidate course, we'll check:
- Grade availability and variance
- Pass rate (57% threshold)
- Number of assignments and modules

In [ ]:
def analyze_course_potential(course_id):
    """
    Analyze a course for analytical potential.
    Returns dict with grades, variance, pass rate, and LMS design metrics.
    """
    result = {
        'course_id': course_id,
        'has_grades': False,
        'n_students_with_grades': 0,
        'grade_variance': 0.0,
        'grade_mean': 0.0,
        'pass_rate': None,
        'n_assignments': 0,
        'n_modules': 0,
        'recommendation': 'SKIP'
    }
    
    # 1. Get enrollments with grades
    enrollments, _ = safe_request(
        f'{API_URL}/api/v1/courses/{course_id}/enrollments',
        headers,
        params={
            'type[]': 'StudentEnrollment',
            'per_page': 100,
            'include[]': 'grades'
        }
    )
    
    if enrollments:
        grades = [
            e['grades'].get('final_score')
            for e in enrollments
            if e.get('grades', {}).get('final_score') is not None
        ]
        
        if len(grades) >= 10:
            result['has_grades'] = True
            result['n_students_with_grades'] = len(grades)
            result['grade_variance'] = np.std(grades)
            result['grade_mean'] = np.mean(grades)
            result['pass_rate'] = sum(1 for g in grades if g >= 57) / len(grades)
    
    # 2. Count assignments
    assignments, _ = safe_request(
        f'{API_URL}/api/v1/courses/{course_id}/assignments',
        headers,
        params={'per_page': 100}
    )
    if assignments:
        result['n_assignments'] = len(assignments)
    
    # 3. Count modules
    modules, _ = safe_request(
        f'{API_URL}/api/v1/courses/{course_id}/modules',
        headers,
        params={'per_page': 100}
    )
    if modules:
        result['n_modules'] = len(modules)
    
    # 4. Determine recommendation
    if result['has_grades'] and result['grade_variance'] > 10:
        if result['n_assignments'] >= 5 and 0.2 <= (result['pass_rate'] or 0) <= 0.8:
            result['recommendation'] = 'HIGH POTENTIAL'
        elif result['n_assignments'] >= 3:
            result['recommendation'] = 'MEDIUM POTENTIAL'
        else:
            result['recommendation'] = 'LOW - Few assignments'
    elif result['has_grades']:
        result['recommendation'] = 'LOW - Low grade variance'
    else:
        result['recommendation'] = 'SKIP - No grades'
    
    return result

print("Course analysis function defined.")

In [ ]:
# Analyze all candidate courses
# WARNING: This makes ~3 API calls per course. Be mindful of rate limits!

analysis_results = []

if len(courses_df) > 0:
    print(f"Analyzing {len(courses_df)} courses...")
    print("(This may take a while due to rate limiting)\n")
    
    for idx, row in tqdm(courses_df.iterrows(), total=len(courses_df), desc="Analyzing"):
        course_id = row['course_id']
        
        analysis = analyze_course_potential(course_id)
        analysis['name'] = row['name']
        analysis['account_id'] = row['account_id']
        analysis['total_students'] = row['students']
        analysis['term_id'] = row['term_id']
        analysis['term_name'] = row['term_name']
        
        analysis_results.append(analysis)
        
        # Show progress for high-potential courses
        if 'HIGH' in analysis['recommendation']:
            print(f"\n  FOUND: {course_id} - {row['name'][:40]}")
            print(f"    Variance: {analysis['grade_variance']:.1f}, Pass Rate: {analysis['pass_rate']:.0%}")
    
    print(f"\n{'='*50}")
    print(f"Analysis complete: {len(analysis_results)} courses analyzed")
else:
    print("No courses to analyze.")

In [ ]:
# Create results DataFrame
if analysis_results:
    results_df = pd.DataFrame(analysis_results)
    
    # Sort by recommendation and grade variance
    results_df['rec_order'] = results_df['recommendation'].map({
        'HIGH POTENTIAL': 1,
        'MEDIUM POTENTIAL': 2,
        'LOW - Few assignments': 3,
        'LOW - Low grade variance': 4,
        'SKIP - No grades': 5
    })
    results_df = results_df.sort_values(['rec_order', 'grade_variance'], ascending=[True, False])
    
    print("Top courses by potential:")
    display(results_df[[
        'course_id', 'name', 'total_students', 'n_students_with_grades',
        'grade_variance', 'pass_rate', 'n_assignments', 'recommendation'
    ]].head(20))
else:
    results_df = pd.DataFrame()
    print("No analysis results.")

## 6. Select Top 5 High-Potential Courses

In [ ]:
# Filter to high and medium potential courses
if len(results_df) > 0:
    high_potential = results_df[results_df['recommendation'].str.contains('HIGH|MEDIUM')].copy()
    
    print(f"High/Medium Potential Courses: {len(high_potential)}")
    print("="*60)
    
    for idx, row in high_potential.head(10).iterrows():
        print(f"\n{row['recommendation']}")
        print(f"  Course ID: {row['course_id']}")
        print(f"  Name: {row['name']}")
        print(f"  Students: {row['total_students']} (with grades: {row['n_students_with_grades']})")
        print(f"  Grade Variance: {row['grade_variance']:.1f}")
        print(f"  Pass Rate: {row['pass_rate']:.1%}" if row['pass_rate'] else "  Pass Rate: N/A")
        print(f"  Assignments: {row['n_assignments']}, Modules: {row['n_modules']}")
        print(f"  Term: {row['term_name']}")
else:
    print("No results to display.")

## 7. Save Results

In [ ]:
# Save all results to CSV
if len(results_df) > 0:
    output_path = '../data/pregrado_discovery_results.csv'
    
    # Select and order columns for output
    output_cols = [
        'course_id', 'name', 'account_id', 'term_id', 'term_name',
        'total_students', 'n_students_with_grades',
        'grade_mean', 'grade_variance', 'pass_rate',
        'n_assignments', 'n_modules', 'recommendation'
    ]
    
    results_df[output_cols].to_csv(output_path, index=False)
    print(f"Results saved to: {output_path}")
    print(f"Total courses: {len(results_df)}")
    
    # Summary statistics
    print(f"\nRecommendation Summary:")
    print(results_df['recommendation'].value_counts())
else:
    print("No results to save.")

## 8. Summary & Next Steps

### Selected Top 5 Courses

| Rank | Course ID | Name | Students | Grade Variance | Pass Rate | Assignments |
|------|-----------|------|----------|----------------|-----------|-------------|
| 1 | | | | | | |
| 2 | | | | | | |
| 3 | | | | | | |
| 4 | | | | | | |
| 5 | | | | | | |

*(Fill in after running the analysis)*

### Next Steps

1. **Review selected courses** - Verify they meet criteria
2. **Extract page views** - Use `intern_exploration_guide.md` ETL code
3. **Document findings** - Update project docs with new courses

### Notes

- Rate limit issues encountered: _(document here)_
- Sub-accounts explored: _(list here)_
- Date of analysis: _(add date)_